# Caratteristiche del notebook

- System_prompt presente = No
- Dataset usato = Hendrycks/math
- Epoche = 3
- Fine Tuning eseguito = Si - Repo -> francescoocurcio/new_llama3.2-3B-math-ftn-math-3epoch_12.5k-sysprompt_no
- Addestrato solo sulle risposte = No

# Importazione delle librerie e definizione delle costanti

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [ ]:
#########################
# IMPORT DELLE LIBRERIE
#########################
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json
import torch
import pandas as pd
pd.set_option('display.max_columns', None)  # Mostra tutte le colonne
pd.set_option('display.width', None)        # Non tronca l'output a una larghezza fissa
pd.set_option('display.max_colwidth', None)

from datasets import load_dataset
from IPython.display import display
from unsloth import FastLanguageModel, to_sharegpt
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

#########################
# COSTANTI
#########################

MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True

OUTPUT_REPO = "francescoocurcio/new_llama3.2-3B-math-ftn-math-3epoch_12.5k-sysprompt_no"
SAVE_DIRECTORY = "/kaggle/working/new_llama3.2-3B-math-ftn-math-3epoch_12.5k-sysprompt_no"

# HuggingFace Login

In [ ]:
!huggingface-cli login --token

# Selezione e configurazione del modello: Llama3.2 3B Instruct

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT
    # token = "hf..." #Use one if using gated models like meta-llama/Llama....
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

#PEFT = Parameter Efficient Fine Tuning
model = FastLanguageModel.get_peft_model( #Modello quantizzato
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Dataset building

In [ ]:
DATASET = "EleutherAI/hendrycks_math"
print("Caricamento del dataset matematico...")

algebra_train = load_dataset(DATASET, 'algebra')['train']
algebra_test = load_dataset(DATASET, 'algebra')['test']

counting_and_probability_train = load_dataset(DATASET, 'counting_and_probability')['train']
counting_and_probability_test = load_dataset(DATASET, 'counting_and_probability')['test']

geometry_train = load_dataset(DATASET, 'geometry')['train']
geometry_test = load_dataset(DATASET, 'geometry')['test']

number_theory_train = load_dataset(DATASET, 'number_theory')['train']
number_theory_test = load_dataset(DATASET, 'number_theory')['test']

intermediate_algebra_train = load_dataset(DATASET, 'intermediate_algebra')['train']
intermediate_algebra_test = load_dataset(DATASET, 'intermediate_algebra')['test']

prealgebra_train = load_dataset(DATASET, 'prealgebra')['train']
prealgebra_test = load_dataset(DATASET, 'prealgebra')['test']

precalculus_train = load_dataset(DATASET, 'precalculus')['train']
precalculus_test = load_dataset(DATASET, 'precalculus')['test']

algebra_train_df = pd.DataFrame(algebra_train)
algebra_test_df = pd.DataFrame(algebra_test)

counting_and_probability_train_df = pd.DataFrame(counting_and_probability_train)
counting_and_probability_test_df = pd.DataFrame(counting_and_probability_test)

geometry_train_df = pd.DataFrame(geometry_train)
geometry_test_df = pd.DataFrame(geometry_test)

number_theory_train_df = pd.DataFrame(number_theory_train)
number_theory_test_df = pd.DataFrame(number_theory_test)

intermediate_algebra_train_df = pd.DataFrame(intermediate_algebra_train)
intermediate_algebra_test_df = pd.DataFrame(intermediate_algebra_test)

prealgebra_train_df = pd.DataFrame(prealgebra_train)
prealgebra_test_df = pd.DataFrame(prealgebra_test)

precalculus_train_df = pd.DataFrame(precalculus_train)
precalculus_test_df = pd.DataFrame(precalculus_test)

dataset_train_df = pd.concat([algebra_train_df, counting_and_probability_train_df, geometry_train_df,
                              number_theory_train_df, intermediate_algebra_train_df,
                              prealgebra_train_df, precalculus_train_df], ignore_index=True)

dataset_test_df = pd.concat([algebra_test_df, counting_and_probability_test_df, geometry_test_df,
                             number_theory_test_df, intermediate_algebra_test_df,
                             prealgebra_test_df, precalculus_test_df], ignore_index=True)

dataset_math_final = pd.concat([dataset_train_df, dataset_test_df], ignore_index=True)

print("Dataset concatenato (train + test):")
dataset_math_final.head()

In [ ]:
from unsloth import to_sharegpt
from datasets import Dataset

dataset = Dataset.from_pandas(dataset_math_final)

dataset = to_sharegpt(
    dataset,
    merged_prompt = "Solve the following problem.[[:\n{problem}]]",
    output_column_name = "solution",
    conversation_extension = 1, #Select more to handle longer conversations
)
print(dataset[0])

## Opzione di aggiunta per il system prompt

In [ ]:
#system_message = {
#    "from": "system",
#    "value": (
#        "You are a math expert. You will be given a mathematical problem to solve. "
#        "Your aim is to first provide a step-by-step explanation of the solution "
#        "(integrating the formulae in LaTeX syntax) and then to conclude with a clear and concise final answer."
#    )
#}

# Aggiungilo all'inizio di ogni conversazione
#def add_system_message(example):
#    conversation = example["conversations"]
#    return {
#        "conversations": [system_message] + conversation
#    }

# Applica la funzione a tutto il dataset
#dataset = dataset.map(add_system_message)
#print(dataset[0])

# Costruzione dataset finale conversazionale

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

print(dataset)
print(dataset[0])

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[0]['text']

In [ ]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.padding_side)
#Anche dopo aver cambiato inizialmente queste operazioni il tokenizer a fronte delle
#operazioni eseguite è come se eseguisse una sorta di riformattazione

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.padding_side)

In [ ]:
text = dataset[0]['text']
tokenized = tokenizer(text, return_tensors="pt", return_attention_mask=True)
input_ids = tokenized["input_ids"][0]

decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids)

for i, (token_id, token_str) in enumerate(zip(input_ids, decoded_tokens)):
    print(f"{i:03d} | Token ID: {token_id.item():>6} | Token: {repr(token_str)}")

# Addestramento del modello tramite LoRA

In [ ]:
from transformers import TrainerCallback

class LossLoggerCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            loss = logs["loss"]
            step = state.global_step
            self.train_losses.append((step, loss))
            print(f"📉 Step {step} - Loss: {loss:.4f}")

from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

loss_callback = LossLoggerCallback()

#model.config.use_cache = False

training_args = SFTConfig(
    do_train                    = True,

    dataset_text_field          = "text",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,

    num_train_epochs            = 3,   # Epoche complete
    #max_steps                   = 10,
    
    learning_rate               = 2e-4,
    lr_scheduler_type           = "linear",
    logging_strategy            = 'steps',
    logging_steps               = 20,
    # 💾 Salvataggio
    save_strategy               = 'epoch',
    #save_steps                  = 200,

    warmup_steps                = 40,

    optim                       = "adamw_8bit",
    seed                        = 42,

    fp16                        = not is_bfloat16_supported(),
    bf16                        = is_bfloat16_supported(),

    weight_decay                = 0.01,
    report_to                   = "none",
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    dataset_num_proc   = 2,
    max_seq_length     = MAX_SEQ_LENGTH,
    train_dataset      = dataset,
    args               = training_args,
    data_collator      = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing            = False,
    callbacks          = [loss_callback]
)

In [ ]:
print("Avvio addestramento LoRA...")
trainer_stats = trainer.train()

# Salvataggio del modello

In [ ]:
!mkdir $SAVE_DIRECTORY

In [ ]:
# PUSH MODELLO LoRA + TOKENIZER su HUGGING FACE
print("🔄 Caricamento del modello e del tokenizer in corso...")
model.push_to_hub(OUTPUT_REPO, token=HF_UNIVERSAL_TOKEN, private=True)
tokenizer.push_to_hub(OUTPUT_REPO, token=HF_UNIVERSAL_TOKEN, private=True)

print(f"✅ Modello caricato correttamente su: {OUTPUT_REPO}\n")
print(f"✅ Tokenizer caricato correttamente su: {OUTPUT_REPO}\n")

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt


# SALVATAGGIO MODELLO LoRA + TOKENIZER
print("💾 Salvataggio del modello e tokenizer...")
trainer.model.save_pretrained(SAVE_DIRECTORY)
tokenizer.save_pretrained(SAVE_DIRECTORY)
print(f"✅ Modello LoRA salvato in: {SAVE_DIRECTORY}")

# SALVATAGGIO LOSS (è stato usato LossLoggerCallback)
loss_path = os.path.join(SAVE_DIRECTORY, "loss_log.csv")

if hasattr(trainer.callback_handler.callbacks[0], "train_losses"):
    print("📊 Esportazione delle loss in CSV...")
    losses = trainer.callback_handler.callbacks[0].train_losses
    loss_df = pd.DataFrame(losses, columns=["step", "loss"])
    loss_df.to_csv(loss_path, index=False)
    print(f"✅ Loss salvate in {loss_path}")
else:
    print("⚠️ Nessuna loss trovata nei callback!")


# VISUALIZZAZIONE LOSS (solo se il file esiste)
if os.path.exists(loss_path):
    print("📈 Visualizzazione della training loss...")
    loss_df = pd.read_csv(loss_path)

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=loss_df, x="step", y="loss", marker="o")

    plt.title("Andamento della Training Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ File {loss_path} non trovato. Salta visualizzazione.")

# Visualizzazione della loss di addestramento

In [ ]:
def crate_loss_chart(json_file):
    # Carica il file JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Estrai i dati di interesse
    log_history = data.get("log_history", [])
    steps = [entry["step"] for entry in log_history]
    losses = [entry["loss"] for entry in log_history]
    
    # Crea un DataFrame
    df = pd.DataFrame({"Step": steps, "Loss": losses})
    
    # Crea il grafico a linee
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df, x="Step", y="Loss", marker='o')
    plt.title("Andamento della Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    
    # Rotazione delle etichette e selezione dei ticks
    plt.xticks(rotation=45)
    plt.locator_params(axis='x', nbins=10)  # Mostra solo 10 ticks sull'asse x
    
    plt.grid(True)
    plt.tight_layout()  # Migliora la disposizione degli elementi
    plt.show()

loss_path = "/kaggle/working/trainer_output/checkpoint-1170/trainer_state.json"
crate_loss_chart(loss_path)